# Tuning an XGBoost Model

The goal of this notebook is to train and evaluate an XGBoost model, comparing it's performance on a holdout set against other types of models (LR, SVC, LDA). 

To ensure reproducibility and consistent evaluation across models, all datasets were **pre-split into cross-val data and holdout data** as below:

| Split type           | CV training file     | Holdout file              | Description                              |
| -------------------- | -------------------- | ------------------------- | ---------------------------------------- |
| **Random**           | `apps_cv_random.csv` | `apps_holdout_random.csv` | Simple random sampling                   |
| **Stratified**       | `apps_cv_strat.csv`  | `apps_holdout_strat.csv`  | Stratified by `TARGET`                   |
| **Multi-Stratified** | `apps_cv_multi.csv`  | `apps_holdout_multi.csv`  | Stratified by `TARGET` + `CODE_GENDER_M` |

Each dataset for cross-validation (`apps_cv_*.csv`) also contains a column, `fold`, with pre-assigned folds from 1-5 using the corresponding splitting method to ensure consistent evaluation. Therefore, no additional splitting is needed inside this notebook -- can simply loop through assigned folds for cross-validation.


In [25]:
import pandas as pd 
import numpy as np 
from xgboost import XGBClassifier
from itertools import product
import time

## Evaluation Functions

Copying over metric functions from `cross_val.ipynb`. Note that in that file, we assigned folds already for each type of splitting to maintain consistent comparison across modeling. So, there will be no explicit splitting in this fild, we will just use the folds already created, but still make it clear what type of splitting was used. 

In [26]:
# METRICS 

def classification_metrics(y_true, y_pred):
    """
    Computes confusion matrix + accuracy, precision, recall, F1, and balanced accuracy.
    """
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)

    # Confusion matrix components
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))

    # Metrics
    acc  = (tp + tn) / max((tp + tn + fp + fn), 1)
    prec = tp / max((tp + fp), 1)
    rec  = tp / max((tp + fn), 1)
    f1   = (2 * prec * rec / max((prec + rec), 1e-12)) if (prec + rec) > 0 else 0.0

    # Specificity (True Negative Rate)
    spec = tn / max((tn + fp), 1)

    # Balanced accuracy
    bal_acc = 0.5 * (rec + spec)

    metrics = {
        "n": len(y_true),
        "tp": tp, "tn": tn, "fp": fp, "fn": fn,
        "acc": acc, "bal_acc": bal_acc, "prec": prec, "rec": rec, "spec": spec,
        "f1": f1
    }
    return metrics

def roc_auc_from_probs(y_true, y_prob):
    
    desc_sort_indices = np.argsort(-y_prob)
    y_true = np.array(y_true)[desc_sort_indices]
    y_prob = np.array(y_prob)[desc_sort_indices]
    pos = np.sum(y_true == 1)
    neg = np.sum(y_true == 0)

    # running totals for TPR/FPR
    tpr = [0.0]
    fpr = [0.0]
    tp = fp = 0
    for i in range(len(y_true)):
        if y_true[i] == 1:
            tp += 1
        else:
            fp += 1
        tpr.append(tp / pos)
        fpr.append(fp / neg)

    # get auc
    auc = np.trapz(tpr, fpr)
    return auc

# Model Development

**Notes:** 
- All evaluation will focus on stratified cross-validation, but we will test the other methods as well. 
- Recall that folds have been pre-assigned to ensure consistency across different model development processes
- For our other models, we have decided to scale + PCA, but this is not necessary for nonlinear tree-based algorithms like XGBoost
    - these can only really hurt XGBoost, so we will not use it here

**Process:**
1. Setting a baseline
    - evaluating an xgb model with all default parameters to build off of
2. Hyperparameter tuning
    - evaluate many different combinations of parameters
    - choose the best set based on average ROC-AUC across all folds
3. Threshold tuning
    - tweak the threshold on the best model to maximize F1 
        - note that roc-auc is not affected by threshold, so we need a different optimizing metric
4. Holdout evaluation
    - evaluate on the corresponding holdout table. the performance here is what we will compare with other models (LR, SVC, LDA)

## 0. Setup


### Read in the data:

Recall that we have different datasets for each type of cross validation. We are currently focusing on the stratified splitting method, but random and multiple-stratification methods are avaiable for testing/comparison.

In [27]:
apps_cv_strat = pd.read_csv("data/apps_cv_strat.csv")
apps_holdout_strat = pd.read_csv("data/apps_holdout_strat.csv")
target_col = 'TARGET'
feature_cols = [col for col in apps_cv_strat.columns if col not in 
                [target_col, 'SK_ID_CURR', 'fold', 'neighbors_target_mean_500']]

Even though we aren't doing PCA and xgboost handles correlated/unecessary features well, we can still simply our model a bit by removing some of them. It will help us run faster and assess feature importance at the end.

In [28]:
corr = apps_cv_strat[feature_cols].corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

print(f"Dropping {len(to_drop)} highly correlated features")

feature_cols_pruned = [f for f in feature_cols if f not in to_drop]

Dropping 35 highly correlated features


### A cross-validation function:

Note that we will tune many xgboost parameters, but some we will set consistently for reproducability. These include:

- eval_metric='auc' : tells xgboost to evaluate performance based on ROC-AUC
- random_state=42 : due to random subsampling in tree building, we fix the random seed to get the same results every run
- n_jobs=-1 : uses all available CPU cores in parallel to speed up training
- tree_method='hist' : a histogram-based algorithm provided by xgboost that can be faster than the default exact method with similar performance
- scale_pos_weight = neg/pos : corrects for class imbalance by upweighting minority class so the model focuses more on them in training. we may change this ratio throughout testing, but it won't be a part of grid searches

These parameters will be constant during every training/evaluation iteration while other parameters are being tuned. 

In [29]:
def cv_xgb(data, feature_cols, target_col, params=None):
    
    if params == None:
        params = {}

    fold_metrics = []
    for f in data.fold.unique():

        # split into train and test based on folds
        train = data[data.fold != f]
        test = data[data.fold == f]
        X_train, y_train = train[feature_cols], train[target_col]
        X_test, y_test = test[feature_cols], test[target_col]

        # calculate counts for class weighting
        pos = (y_train == 1).sum()
        neg = (y_train == 0).sum()
        balanced_weight = neg / max(pos, 1)

        # fit model with specified params
        model = XGBClassifier(eval_metric='auc', 
                              random_state=42, 
                              n_jobs=-1, 
                              tree_method='hist',
                              scale_pos_weight=balanced_weight,
                              **params
                            )
        model.fit(X_train, y_train)
        
        # get predictions (probablities and decisions)
        y_prob = model.predict_proba(X_test)[:, 1]
        y_pred = (y_prob >= 0.5).astype(int)

        # calculate classification metrics from previously defined functions
        metrics = classification_metrics(y_test, y_pred)
        metrics['roc_auc'] = roc_auc_from_probs(y_test, y_prob)
        metrics['fold'] = int(f)

        # add to list of all fold metrics
        fold_metrics.append(metrics)

    # return results in dataframe
    return pd.DataFrame(fold_metrics).sort_values("fold").reset_index(drop=True)

### A grid search function:

Tests every combination of hyperparameters -- very slow/inefficient, so consider size of grid.

In [30]:
def grid_search_xgb(data, feature_cols, target_col, param_grid):
    
    # get all possible combinations of parameters
    keys = list(param_grid.keys())
    combos = [dict(zip(keys, v)) for v in product(*param_grid.values())]

    # initialize stuff for tracking and results
    results = []
    total = len(combos)
    start = time.time()
    next_checkpoint = 5 
    best_roc_auc = 0
    best_params = None

    # evaluate every possible combo
    for i, params in enumerate(combos, 1):

        # run cross validation and store results
        fold_results = cv_xgb(data, feature_cols, target_col, params)
        mean_roc_auc = fold_results["roc_auc"].mean()

        results.append({
            'params': params,
            'mean_roc_auc': mean_roc_auc,
            'mean_f1': fold_results['f1'].mean(),
            'mean_acc': fold_results['acc'].mean(),
            'mean_bal_acc': fold_results['bal_acc'].mean(),
            'mean_prec': fold_results['prec'].mean(),
            'mean_rec': fold_results['rec'].mean(),
        })

        # tracker for updates
        if  mean_roc_auc > best_roc_auc:
            best_roc_auc =  mean_roc_auc
            best_params = params

        # print progress checkpoints
        pct_done = (i/total)*100
        elapsed = time.time() - start
        if pct_done >= next_checkpoint or i == total:
            print(f"{i}/{total} ({pct_done:5.1f}% in {elapsed/60:.1f} mins) | Best ROC-AUC: {best_roc_auc:.4f} | Best Params: {best_params}")
            next_checkpoint += 5


    
    return pd.DataFrame(results).sort_values("mean_roc_auc", ascending=False).reset_index(drop=True)

## 1. Setting a Baseline

Fitting an XGBoost model with default parameters to understand baseline predictive power and what we can build on.

In [31]:
baseline_results = cv_xgb(apps_cv_strat, feature_cols, target_col, params=None) # send no parameters
baseline_results

,n,tp,tn,fp,fn,acc,bal_acc,prec,rec,spec,f1,roc_auc,fold
0,49156,2348,35419,9768,1621,0.768309,0.687708,0.193793,0.591585,0.783832,0.291949,0.763353,1
1,49156,2362,35058,10129,1607,0.761250,0.685477,0.189096,0.595112,0.775843,0.286999,0.758464,2
2,49156,2317,35387,9800,1652,0.767027,0.683449,0.191219,0.583774,0.783123,0.288077,0.759206,3
3,49155,2416,35363,9823,1553,0.768569,0.695664,0.197402,0.608718,0.782610,0.298124,0.766919,4
4,49154,2351,35477,9709,1617,0.769581,0.688811,0.194942,0.592490,0.785133,0.293362,0.763787,5


Precision, recall, and F1 are all quite low, while ROC-AUC is relatively strong. This indicates that the model is doing a good job ranking applicants from low-risk to high-risk, as in assigning higher probabilities to true positives, but the actual 0/1 decisions at the default threshold of 0.5 are poor (likely due to class imbalance).

Therefore, we expect significant improvement after threshold tuning at the end. This is also why we focus on ROC-AUC during hyperparameter tuning -- as long as the model’s ability to rank cases is good (high ROC-AUC), we can later adjust the threshold to manipulate precision/recall/F1.

## 2. Hyperparameter Tuning

Now, finding hyperparameters that maximize ROC-AUC (while keeping an eye on other metrics). 

XGBoost has a lot of parameters, many of which make a big impact on predictions, so the method of selecting the best ones is more complicated than the other linear models. A simple grid search over a huge parameter grid will take forever, considering just one cross-validated iteration of the baseline model took 15+ seconds. 

Therefore, a smart approach may be to do a multi-step grid search. There are two ways we could do this:

1. Tree structure --> learning dynamics
    - first a grid search on parameters that affect the shape of the trees essentially (max_depth, subsample, etc.)
    - then a grid search on paramaters that affect how the model learns (learning_rate, estimators, etc.)
2. Wide range, big step size --> small range, small step size
    - a grid search on *all* parameters with a very wide range for each parameter
    - then a more granular search on the best parameters to find exact optimal values

We will start with the first structural approach, but then may incorporate the second approach at some points as well. 

### First Pass:

Optimizing tree structure.

In [32]:
param_grid = {

    # fix learning dynamics this pass
    "learning_rate": [0.05], # low learning rate at first to establish reliable tree structure
    "n_estimators": [500], 

    # tree structure parameters to test
    "max_depth": [3, 4, 5, 6, 7],
    "min_child_weight": [1, 3, 5, 8, 12],
    "subsample": [0.7, 0.85, 1.0],
    "colsample_bytree": [0.7, 0.85, 1.0],
}

search1_results = grid_search_xgb(apps_cv_strat, feature_cols, target_col, param_grid)

12/225 (  5.3% in 7.5 mins) | Best ROC-AUC: 0.7790 | Best Params: {'learning_rate': 0.05, 'n_estimators': 500, 'max_depth': 3, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.85}
23/225 ( 10.2% in 14.7 mins) | Best ROC-AUC: 0.7790 | Best Params: {'learning_rate': 0.05, 'n_estimators': 500, 'max_depth': 3, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.85}
34/225 ( 15.1% in 21.5 mins) | Best ROC-AUC: 0.7791 | Best Params: {'learning_rate': 0.05, 'n_estimators': 500, 'max_depth': 3, 'min_child_weight': 8, 'subsample': 0.7, 'colsample_bytree': 0.85}
45/225 ( 20.0% in 27.5 mins) | Best ROC-AUC: 0.7791 | Best Params: {'learning_rate': 0.05, 'n_estimators': 500, 'max_depth': 3, 'min_child_weight': 12, 'subsample': 0.7, 'colsample_bytree': 0.85}
57/225 ( 25.3% in 34.6 mins) | Best ROC-AUC: 0.7810 | Best Params: {'learning_rate': 0.05, 'n_estimators': 500, 'max_depth': 4, 'min_child_weight': 3, 'subsample': 0.7, 'colsample_bytree': 0.7}
68/225 ( 30.2% in 41.3 min

In [34]:
# save so we dont have to run it all again
search1_results.to_csv('results/xgb_search1_results.csv')

In [47]:
search1_results.sort_values(by='mean_roc_auc', ascending=False)[
    ['params', 'mean_roc_auc', 'mean_prec', 'mean_rec', 'mean_f1']].head(10)

,params,mean_roc_auc,mean_prec,mean_rec,mean_f1
0,"{'learning_rate': 0.05, 'n_estimators': 500, '...",0.781358,0.184567,0.685850,0.290855
1,"{'learning_rate': 0.05, 'n_estimators': 500, '...",0.781235,0.192193,0.661913,0.297880
2,"{'learning_rate': 0.05, 'n_estimators': 500, '...",0.781106,0.184413,0.684237,0.290517
3,"{'learning_rate': 0.05, 'n_estimators': 500, '...",0.781089,0.184827,0.684439,0.291051
4,"{'learning_rate': 0.05, 'n_estimators': 500, '...",0.781088,0.192360,0.661510,0.298040
5,"{'learning_rate': 0.05, 'n_estimators': 500, '...",0.781088,0.184615,0.684943,0.290832
6,"{'learning_rate': 0.05, 'n_estimators': 500, '...",0.781080,0.184300,0.686051,0.290543
7,"{'learning_rate': 0.05, 'n_estimators': 500, '...",0.781050,0.191841,0.664533,0.297723
8,"{'learning_rate': 0.05, 'n_estimators': 500, '...",0.781029,0.184236,0.683632,0.290245
9,"{'learning_rate': 0.05, 'n_estimators': 500, '...",0.781008,0.184761,0.684539,0.290977


It looks like we found decent lift (+0.02 ROC-AUC) from tuning the tree structure. We also virtually identical scores at the top, not just one outlier, which is a good sign we found an optimal structure -- if these top scores had similar parameters, that would be further proof:

In [58]:
params_df = search1_results['params'].apply(pd.Series)
results = pd.concat([search1_results.drop(columns='params'), params_df], axis=1)
results.sort_values(by='mean_roc_auc', ascending=False)[
    ['mean_roc_auc', 'max_depth', 'min_child_weight', 'subsample', 'colsample_bytree']].head()

,mean_roc_auc,max_depth,min_child_weight,subsample,colsample_bytree
0,0.781358,4.0,8.0,0.70,0.7
1,0.781235,5.0,8.0,0.85,1.0
2,0.781106,4.0,12.0,0.70,1.0
3,0.781089,4.0,12.0,0.85,1.0
4,0.781088,5.0,8.0,0.70,1.0


The top 5 models by ROC-AUC have very small changes in parameters -- The optimal region occurs with moderate `depth` (4–5), high `min_child_weight` (8–12), and subsampling between 0.7–0.85, suggesting strong generalization and reproducability. We also don't see any sitting on a single edge, like if all the best performers were at the highest depth, so there is no reason to believe we are missing anything beyond our grid. 

These parameters will be fixed while tuning learning dynamics in the next phase:

- max_depth = 4
- min_child_weight = 8
- subsample = 0.7
- colsample_bytree = 1

These were the parameters of the best model by ROC-AUC, *except* for `colsample_bytree` because four out of the top five models had this parameter set to 1, so I felt it was safest to keep it as that, even though the #1 model was 0.7.

### Second Pass: Learning Dynamics

Now that we have a reliable tree structure, we will focus on the parameters that effect how the model learns. 

I think in this case it makes more sense to do the wide-then-narrow search approach because the learning parameters are continuous and small changes can make a big difference. Therefore, we will first find the appropriate scale of each parameter, and then do a narrow search over a smaller range to find more exact optimal parameters. Also, the difference from the best to 2nd (or even 5th) best is so small it's negligble.

**Finding Scale of Learning Dynamics:**

In [ ]:
param_grid = {

    # fix tree structure to optimal parameters found in pass 1
    "max_depth": [4],
    "min_child_weight": [8],
    "subsample": [0.7],
    "colsample_bytree": [1],

    # wide grid to find scale of learning dynamics
    "learning_rate": [0.01, 0.03, 0.05, 0.07, 0.10],
    "n_estimators": [300, 500, 800, 1200],
    "gamma": [0.0, 0.1, 0.2],
    "reg_alpha": [0.0, 0.5, 1.0],
    "reg_lambda": [1.0, 2.0, 5.0],
}

search2_results = grid_search_xgb(apps_cv_strat, feature_cols, target_col, param_grid)